In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [6]:
from sympy import sqrt, cos, sin,  cosh,  sinh, Add
from sympy import Symbol, S, oo, Rational, Function, pi , factorial, O, symbols, lambdify, I
from sympy import Derivative, series, limit, Matrix, simplify, collect, solve, Poly, poly, diff, nsolve, transpose, factor_terms
from sympy import latex, init_printing
from sympy.core.numbers import NegativeOne, One, Zero
from sympy.printing.latex import print_latex

import numpy as np
import pandas as pd
from tqdm import tqdm
import ray
import time

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import defaultdict
from itertools import product, combinations
from functools import reduce
from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList, GenerateNDimCoordinateGrid
from acchamiltoniansandmatrices.Hamiltonians.LatticeElementHamiltonians import HamDrift6D, HamQuad6D
from acchamiltoniansandmatrices.Hamiltonians.Operators import Poisson
from acchamiltoniansandmatrices.LieMaps.Poisson import PoissonBracket
from acchamiltoniansandmatrices.LieMaps.LieOperator import LieOperator
from acchamiltoniansandmatrices.Utils.Plotting import multi_countour_plot
from acchamiltoniansandmatrices.Factorization.Factorization import truncate, taylorize, dragt_finn_factorization
from acchamiltoniansandmatrices.Utils.Polynomials import getpoly, RotationMatrix2D, explicitCheck, geteigenvects
from acchamiltoniansandmatrices.Matrices.NumpyMatrices import RnpQuad6DChroma
from acchamiltoniansandmatrices.Matrices.SymbMatrices import RsymbQuad6DChroma
from acchamiltoniansandmatrices.Integrators import integrator
from acchamiltoniansandmatrices.Tracking.LinearMatrixTracking import nestList

from acchamiltoniansandmatrices.LieMaps.BCH import generate_bch_expression

In [4]:
x, px, y, py, z, delta = symbols('x p_x y p_y z delta', real=True)
eps, h , k0, k1, k2, sigma, betag = symbols('epsilon h k_0 k_1 k_2 sigma beta_gamma', real=True)
L, beta0, gamma0 = symbols(" L beta_0 gamma_0", real=True, positive=True)
x0,px0 = symbols('x_0 px_0') # to not overwrite anything
p = symbols("p")

In [105]:
H1 = px**2+x**2
H2 = px**2

# Comparing old_BCH with new

In [90]:
X = Function("X")(x,px)
Y = Function("Y")(x,px)

Lo1 = LieOperator(X,[x],[px]) 
Lo2 = LieOperator(Y,[x],[px])

## order 2

In [91]:
Lo1.BCH(Lo2,2).ham

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2

In [92]:
generate_bch_expression(order=2,coords=[x],mom=[px])

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2

## order 3

In [93]:
Lo1.BCH(Lo2,3).ham.expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12

In [94]:
generate_bch_expression(order=3,coords=[x],mom=[px]).expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12

## order 4

In [95]:
Lo1.BCH(Lo2,4).ham.expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24

In [96]:
generate_bch_expression(order=4,coords=[x],mom=[px]).expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24

## order 5

In [97]:
Lo1.BCH(Lo2,5).ham.expand(commutator=True)

X(x, p_x) + Y(x, p_x) - PoissonBracket(X(x, p_x)/720, PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x))) - PoissonBracket(X(x, p_x)/360, PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), Y(x, p_x))) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 + PoissonBracket(Y(x, p_x)/720, PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), Y(x, p_x))) - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), Y(x, p_x))/360 + PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x)), X(x, p_x))/120 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x))

In [98]:
generate_bch_expression(order=5,coords=[x],mom=[px]).expand(commutator=True)

X(x, p_x) + Y(x, p_x) + PoissonBracket(X(x, p_x), Y(x, p_x))/2 - PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x))/12 + PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x))/12 - PoissonBracket(-PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), -PoissonBracket(X(x, p_x), Y(x, p_x)))/120 - PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x))/24 - PoissonBracket(-PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), Y(x, p_x)), -PoissonBracket(X(x, p_x), Y(x, p_x)))/360 + PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), X(x, p_x))/720 + PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), X(x, p_x)), Y(x, p_x))/180 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p_x)), X(x, p_x)), Y(x, p_x)), Y(x, p_x))/180 - PoissonBracket(PoissonBracket(PoissonBracket(PoissonBracket(X(x, p_x), Y(x, p

# Practical example

In [106]:
Lo1.BCH(Lo2,1).ham.subs(X,H2).subs(Y,H1).doit()

2*p_x**2 + x**2

In [107]:
generate_bch_expression(order=1,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1)

2*p_x**2 + x**2

In [108]:
Lo1.BCH(Lo2,2).ham.subs(X,H2).subs(Y,H1).doit()

2*p_x**2 - 2*p_x*x + x**2

In [109]:
generate_bch_expression(order=2,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1).doit()

2*p_x**2 - 2*p_x*x + x**2

In [110]:
Lo1.BCH(Lo2,3).ham.subs(X,H2).subs(Y,H1).doit()

2*p_x**2 - 2*p_x*x + 5*x**2/3

In [111]:
generate_bch_expression(order=3,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1).doit()

2*p_x**2 - 2*p_x*x + 5*x**2/3

In [112]:
Lo1.BCH(Lo2,4).ham.subs(X,H2).subs(Y,H1).doit()

2*p_x**2 - 10*p_x*x/3 + 5*x**2/3

In [113]:
generate_bch_expression(order=4,coords=[x],mom=[px]).expand(commutator=True).subs(X,H2).subs(Y,H1).doit()

2*p_x**2 - 2*p_x*x + 5*x**2/3

In [114]:
Lo1.BCH(Lo2,5).ham.subs(X,H2).subs(Y,H1).doit()

98*p_x**2/45 - 10*p_x*x/3 + 107*x**2/45

In [115]:
generate_bch_expression(order=5,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

98*p_x**2/45 - 10*p_x*x/3 + 107*x**2/45

In [116]:
generate_bch_expression(order=6,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

98*p_x**2/45 - 214*p_x*x/45 + 107*x**2/45

In [118]:
generate_bch_expression(order=7,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

2378*p_x**2/945 - 214*p_x*x/45 + 433*x**2/135

In [121]:
generate_bch_expression(order=8,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

2378*p_x**2/945 - 866*p_x*x/135 + 433*x**2/135

In [122]:
generate_bch_expression(order=9,coords=[x],mom=[px]).subs(X,H2).subs(Y,H1).doit()

14258*p_x**2/4725 - 866*p_x*x/135 + 19891*x**2/4725

In [123]:
14258/4725

3.0175661375661376

In [119]:
2378/945

2.516402116402116

In [120]:
98/45

2.1777777777777776